<a href="https://colab.research.google.com/github/SoumyaShreeram/Microlensing_with_NeuralNets/blob/master/04_Neural_Network_Designs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building the architectures for the Neural Networks

The following script defines three different Neural Nets:
1. **CNN1**: CNN without batch normalization
2. **CNN2**: CNN with batch normalization
2. **ResNet** 

Author: Soumya Shreeram
Script adapted from: Millon Martin & Kevin Müller
Date: 16th March 2020

In [0]:
from google.colab import drive
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

%tensorflow_version 2.x
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import (Input, Dense, Conv1D, MaxPooling1D,
                          Dropout, Flatten, BatchNormalization)
from keras import optimizers
from keras import regularizers

TensorFlow 2.x selected.


Using TensorFlow backend.


### 2. Defines a modified pooling layer that outputs a 3D tenson

Note: the `keras.layers.GLobalMaxPooling2D` returns a 2D tensor of shape: `(batch_size, channels)`. However, this pooling layer outputs a shape: `(batch_size, channels, rows or cols)` based on the input of data format.

In [0]:
class GlobalMaxPoolingSp2D(Layer):
    """Global max pooling operation for spatial data along a single dimension.
    # Arguments
        sqash_dim: A scalar
            1: Find the maximum along the columns (the output doesn't the row dimension)
            2: Find the maximum along the rows (the output doesn't the column dimension)
            Defaults: squash_dim=2
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
    # Input shape
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, rows, cols, channels)`
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, rows, cols)`
    # Output shape
        - If `data_format='channels_last'`:
            3D tensor with shape:
            `(batch_size, rows or cols, channels)`
        - If `data_format='channels_last'`:
            3D tensor with shape:
            `(batch_size, channels, rows or cols)`
    """
    
  
    def __init__(self, squash_dim=2, data_format=None, **kwargs):
        if data_format is None:
          data_format = K.image_data_format()
        data_format = data_format.lower()
        if data_format not in {'channels_first', 'channels_last'}:
          raise ValueError('The `data_format` argument must be one of '
                           '"channels_first", "channels_last". Received: ' +
                           str(value))
        self.data_format = data_format
        
        self.input_spec = InputSpec(ndim=4)
        self.squash_dim = squash_dim
        super(GlobalMaxPoolingSp2D, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3-self.squash_dim], input_shape[3])
        else:
            return (input_shape[0], input_shape[1], input_shape[4-self.squash_dim])

    def call(self, inputs):
        if self.data_format == 'channels_last':
            return K.max(inputs, axis=self.squash_dim)
        else:
            return K.max(inputs, axis=self.squash_dim+1)
      
    def get_config(self):
        config = {'data_format': self.data_format}
        base_config = super(GlobalMaxPoolingSp2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

General functions used to add:
*   convolutional layers
*   max pooling layers
*   fully connected layers

In [0]:
def addConvPoolingLayers(model, num_pix, num_filters, kernel_size, maxpoolsize):
  """
  Function to add convolutional and max pooling layers to the model
  """
  in_shape = (num_pix, None, 1) 
  kernel_size = (kernel_size, 1)
  strides = (1,1)
  # adds convolutional layers
  model.add(Conv2D(num_filters, kernel_size, strides=strides, \
                  padding='same', input_shape = in_shape, activation='relu'))
        
  # adds pooling layers
  if maxpoolsize is not None:
    poolsize = (maxpoolsize,1)
    model.add(MaxPooling2D(pool_size=poolsize, strides =(maxpoolsize,1)))
  return model

def addFullyConnectedLayers(model, num_hidden_nodes, dropout_ratio, r_0):
  """
  Function to add fully connected layers to the model
  @num_hidden_nodes :: no. of nodes in the layers prior to the output layer
  @dropout_ratio :: weight constrain in the dropout layers
  @r_0 :: len(r_0) defines the no. of nodes in the output layer
  """
  # post CNN; fully connected layers
  for i, nodes in enumerate(num_hidden_nodes):
    model.add(Dense(nodes, activation='sigmoid'))
    model.add(Dropout(dropout_ratio))

  # final output layer
  model.add(Dense(len(r_0), activation='softmax'))
  return model

### 3.1 Convolutional Neural Network: Design 1

The following model used the `keras` Sequential models

In [0]:
def buildModelCNN1(num_pix, num_filters, kernel_size, maxpoolsize, num_hidden_nodes,\
              r_0, optim):
  """
  Function to build the model architecture, set optimizers and compile the model 
  @num_pix :: used to define the shape of the input layer
  @num_filters :: arr with the ascending order of filters in the conv2D layers
  @kernel_size :: arr with the kernel sizes for the corresponding conv2D layers
  @maxpoolsize :: arr with the pool sizes for the corresponding conv2D layers
  @num_hidden_nodes :: no. of nodes in the FNN layer right after the CNN
  @r_0 :: array of all the scale radii
  @opt :: optimizer-type for the Neural Net

  @Returns:: model
  """
  model = Sequential()
  
  # adding convolutional and pooling layers
  model = addConvPoolingLayers(model, num_pix, num_filters, kernel_size, maxpoolsize)
  model.add(GlobalMaxPoolingSp2D())
  model.add(Flatten())
  
  # fully connected layers; added dropout these layer with weight constraint
  model = addFullyConnectedLayers(model, num_pix, num_hidden_nodes, dropout_ratio, r_0)

  # optimizer 
  sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=False)

  # compilation for a multi-category classification problem
  model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

### 3.2 Convolutional Neural Network: Design 2

This CNN accounts for batch normalization unlike the first case. Additionally, used Keras functional API for more flexibitity.

In [0]:
def builfModelCNN2(num_filter, kern_size, maxpoolsize, num_hidden, \
                   dropout_ratio, shortcut_link, batch_norm, length_traj):
  """
  Function to build the model architecture, set optimizers and compile the model 
  @length_traj :: used to define the shape of the input layer
  @num_filters :: arr with the ascending order of filters in the conv2D layers
  @kernel_size :: arr with the kernel sizes for the corresponding conv2D layers
  @maxpoolsize :: arr with the pool sizes for the corresponding conv2D layers
  @num_hidden_nodes :: no. of nodes in the FNN layer right after the CNN
  @r_0 :: array of all the scale radii
  @opt :: optimizer-type for the Neural Net

  @Returns:: model
  """
  visible = Input(shape=(length_traj, None, 1))
  
  # adding convolutional and pooling layers
  for i in range(len(num_filter)):
    kernel_size = (kernel_size[i], 1)
    strides = (1,1)
    hidden1 = Conv2D(num_filters, kernel_size, strides=strides, padding='same',
                     activation='relu')(visible)

    # batch normalization (nomalizes the inputted layer)
    hidden1 = BatchNormalization()(hidden1)  
    hidden1 = Activation('selu')(hidden1)
    
    #              
  model.add(GlobalMaxPoolingSp2D())
  model.add(Flatten())
  
  # fully connected layers; added dropout these layer with weight constraint
  model = addFullyConnectedLayers(model, num_hidden_nodes, dropout_ratio, r_0)

  # optimizer 
  sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=False)

  # compilation for a multi-category classification problem
  model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
  return model
  return model

### 3.2 Residual neural network: ResNet

In [0]:
def buildModelResNet(num_filter, kern_size, n_block, maxpoolsize, num_hidden, \
                dropout_ratio, batch_norm = True, optim):
  """
  Function to build the model architecture, set optimizers and compile the model 
  @num_pix :: used to define the shape of the input layer
  @num_filters :: arr with the ascending order of filters in the conv2D layers
  @kernel_size :: arr with the kernel sizes for the corresponding conv2D layers
  @maxpoolsize :: arr with the pool sizes for the corresponding conv2D layers
  @num_hidden_nodes :: no. of nodes in the FNN layer right after the CNN
  @dropout_ratio :: weight constrain on the dropout layer of the FNN 
  @r_0 :: array of all the scale radii
  @opt :: optimizer-type for the Neural Net

  @Returns:: model
  """
  model = Sequential()
  model.add(AveragePooling1D(pool_size=5))
  
  # adding convolutional and pooling layers
  model = addConvPoolingLayers(model, num_filters, kernel_size, maxpoolsize)
  # some output shortcut trickery

  if batch_norm: 
    model.add(layers.BatchNormalization())
  model.add(GlobalMaxPoolingSp2D())
  model.add(Flatten())
  
  # fully connected layers; added dropout these layer with weight constraint
  model = addFullyConnectedLayers(model, num_hidden_nodes, dropout_ratio, r_0)

  # optimizer 
  sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=False)

  # compilation for a multi-category classification problem
  model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
  return model